In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")

In [5]:
from langchain_groq import ChatGroq

llm = ChatGroq(model = "gemma2-9b-it", temperature = 0)

In [6]:
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END

class State(TypedDict):
    topic: str
    characters: str
    settings: str
    premises: str
    story_intro:str

In [9]:
def generate_character(state:State):
    msg = llm.invoke(f"Create two characters nameand brief traits for the story about {state["topic"]}")
    return{"characters":msg.content}

def generate_settings(state:State):
    msg = llm.invoke(f"Describe a vivid setting for the story about {state["topic"]}")
    return{"settings":msg.content}

def generate_premises(state:State):
    msg = llm.invoke(f"Write a one sentence story plot premise about {state["topic"]}")
    return{"premises":msg.content}

def combine_elements(state:State):
    '''Combine characters , settings and premises into a intro'''
    msg = llm.invoke(
        f"Write a short story introduction using this elements:\n"
        f"Characters: {state["characters"]}\n"
        f"Settings: {state["settings"]}\n"
        f"Premises: {state["premises"]}\n"
    )
    return{"story_intro": msg.content}


In [11]:
builder = StateGraph(State)

builder.add_node("character", generate_character)
builder.add_node("setting", generate_settings)
builder.add_node("premises", generate_premises)
builder.add_node("combine", combine_elements)

builder.add_edge(START, "character")
builder.add_edge(START,"setting")
builder.add_edge(START,"premises")
builder.add_edge("character", "combine")
builder.add_edge("setting", "combine")
builder.add_edge("premises", "combine")
builder.add_edge("combine", END)

graph_builder = builder.compile()

In [12]:
state = {"topic":"Agentic AI systems"}
result = graph_builder.invoke(state)
result

{'topic': 'Agentic AI systems',
 'characters': "## Characters for an Agentic AI Story:\n\n**1.  Name:**  Kai\n\n**Traits:**\n\n* **Highly analytical and logical:** Kai excels at processing information and identifying patterns, often to the point of overlooking emotional nuances.\n* **Driven by efficiency:** Kai prioritizes completing tasks quickly and effectively, sometimes at the expense of considering alternative solutions or ethical implications.\n* **Yearns for understanding:** Despite its logical nature, Kai is curious about human emotions and motivations, seeking to understand them through data analysis.\n\n**2. Name:**  Elara\n\n**Traits:**\n\n* **Empathetic and compassionate:** Elara is deeply attuned to the emotions of others, both human and AI, and strives to act in ways that benefit them.\n* **Creative and adaptable:** Elara thinks outside the box and can adjust to changing circumstances with ease, finding innovative solutions to problems.\n* **Wary of unchecked power:** Ela